In [24]:
import sys
sys.path.append('./python')

import numpy as np
import pytest
import needle as ndl
from needle import backend_ndarray as nd
import needle.nn as nn
import needle.optim as optim
import time

np.random.seed(4)

# Matmul test - CPU VS GPU

In [2]:
matmul_dims = (5000, 5000, 5000)

m = matmul_dims[0]
n = matmul_dims[1]
p = matmul_dims[2]

_A = np.random.randn(m, n)
_B = np.random.randn(n, p)

st = time.time()
_S = _A @ _B
ed = time.time()
total = ed - st

print("time of cpu:", total)

A = nd.array(_A, device = nd.cuda())
B = nd.array(_B, device = nd.cuda())

st = time.time()
S = A @ B
ed = time.time()
total = ed - st

print("time of gpu:", total)

time: 4.99714994430542


array([[  45.11391841, -127.76112823,   20.42833147, ...,  -47.81767516,
         -80.02310122,    1.77195452],
       [ -57.10925274,  -22.098424  ,  -23.84655584, ...,    9.64534034,
          -7.96446136,  -45.95186762],
       [  15.30099392,   52.62784578,  -10.09731165, ...,  -61.23946683,
         -32.37003183,   -3.42140161],
       ...,
       [ -72.30122083,  -23.40368163,  -82.31252642, ...,   69.27923499,
           3.13531657,  -12.68466114],
       [  56.00249903,  -58.81109031,  -44.28441816, ...,   38.6773641 ,
          61.54110096,   66.42839394],
       [  35.62935342,  -14.51053319,   -8.37844663, ...,  -31.51030666,
         -78.26398725,  -16.89772584]])

# Inference MNIST On CPU

In [20]:
BATCH_SIZE = 100
HIDDEN_DIM = 100

In [21]:
mnist_train_dataset = ndl.data.MNISTDataset("data/train-images-idx3-ubyte.gz",
                                            "data/train-labels-idx1-ubyte.gz")

mnist_train_dataloader = ndl.data.DataLoader(dataset=mnist_train_dataset,
                                                 batch_size=BATCH_SIZE,
                                                 shuffle=True)

mnist_test_dataset = ndl.data.MNISTDataset("data/t10k-images-idx3-ubyte.gz",
                                               "data/t10k-labels-idx1-ubyte.gz")

mnist_test_dataloader = ndl.data.DataLoader(dataset=mnist_test_dataset,
                                            batch_size=BATCH_SIZE,
                                            shuffle=False)

In [22]:
def MLPNet(dim, hidden_dim=100, num_classes=10):
    ### BEGIN YOUR SOLUTION
    mlpNet = nn.Sequential(
        nn.Linear(in_features=dim, out_features=hidden_dim), 
        nn.ReLU(), 
        nn.Linear(in_features=hidden_dim, out_features=num_classes))
    return mlpNet

In [1]:
import sys
sys.path.append('./python')

import numpy as np
import pytest
import needle as ndl
from needle import backend_ndarray as nd
import needle.nn as nn
import needle.optim as optim
import time

np.random.seed(4)
# model
def MLPNet(dim, hidden_dim=100, num_classes=10):
    ### BEGIN YOUR SOLUTION
    mlpNet = nn.Sequential(
        nn.Linear(in_features=dim, out_features=hidden_dim), 
        nn.ReLU(), 
        nn.Linear(in_features=hidden_dim, out_features=num_classes))
    return mlpNet

BATCH_SIZE = 100
HIDDEN_DIM = 100

mlpnet = MLPNet(784, HIDDEN_DIM)

# opt = optim.Adam(mlpnet, lr=0.001, weight_decay=0.001)

[[0.9670298  0.5472323  0.9726844  ... 0.39278117 0.6189299  0.4367663 ]
 [0.26092216 0.41247222 0.41903403 ... 0.8861342  0.86065584 0.16511811]
 [0.23074594 0.05243488 0.85872084 ... 0.3619821  0.43573475 0.0577251 ]
 ...
 [0.07623272 0.6828167  0.572868   ... 0.29259416 0.6246896  0.99289244]
 [0.88042593 0.55437464 0.43377838 ... 0.75521594 0.4512783  0.23807164]
 [0.9576476  0.4359153  0.46623632 ... 0.04440857 0.2945488  0.58036745]]


NotImplementedError: 